In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('./dataset/kuesioner.xlsx')
df

,DM,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10
0,DM1,4,4,5,5,5,4,5,4,5,4
1,DM2,5,5,5,5,5,5,5,5,5,5
2,DM3,4,5,3,3,4,4,4,5,4,3
3,DM4,5,5,5,5,5,5,5,5,5,5
4,DM5,4,4,2,3,4,4,3,4,4,3
5,DM6,4,4,4,4,5,5,4,5,5,4
6,DM7,5,4,4,4,3,5,3,5,5,5
7,DM8,5,3,5,3,4,4,3,3,3,5
8,DM9,4,5,5,3,4,5,4,5,5,5
9,DM10,3,4,3,5,3,3,3,3,4,5


In [3]:
df['yi'] = df.iloc[:,1:11].sum(axis=1)
df['yi^2'] = (df['yi']**2)
sig_xi = df.iloc[:,1:].sum(axis=0)
sig_xi_square = (df.iloc[:,1:11] ** 2).sum(axis=0)
sig_xy = (df.iloc[:, 1:11].multiply(df['yi'], axis=0)).sum()

df1 = pd.DataFrame([sig_xi, sig_xi_square, sig_xy],
                            index=['Σ_xi', 'Σ_xi^2', 'Σ_x*y'],
                            columns=df.columns[1:11])

# Menambahkan kolom "DM" untuk nama baris
df1['DM'] = ['Σ_xi', 'Σ_xi^2', 'Σ_x*y']
# Menyesuaikan urutan kolom agar "DM" tetap di depan
df1 = df1[['DM'] + list(df1.columns[:-1])]
# Menggabungkan dataframe asli dengan baris tambahan menggunakan pd.concat()
df_corr = pd.concat([df, df1], ignore_index=True)
df_corr 

,DM,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,yi,yi^2
0,DM1,4,4,5,5,5,4,5,4,5,4,45.0,2025.0
1,DM2,5,5,5,5,5,5,5,5,5,5,50.0,2500.0
2,DM3,4,5,3,3,4,4,4,5,4,3,39.0,1521.0
3,DM4,5,5,5,5,5,5,5,5,5,5,50.0,2500.0
4,DM5,4,4,2,3,4,4,3,4,4,3,35.0,1225.0
5,DM6,4,4,4,4,5,5,4,5,5,4,44.0,1936.0
6,DM7,5,4,4,4,3,5,3,5,5,5,43.0,1849.0
7,DM8,5,3,5,3,4,4,3,3,3,5,38.0,1444.0
8,DM9,4,5,5,3,4,5,4,5,5,5,45.0,2025.0
9,DM10,3,4,3,5,3,3,3,3,4,5,36.0,1296.0


In [4]:
rtabel = 0.514

def pearson_correlation_fixed(df, col_x, col_y):

    # Extract values for xi and yi
    x_values = df[col_x].values
    y_values = df.iloc[:,1:11].sum(axis=1)
    df['yi'] = y_values

    # n is the number of data points
    n = len(x_values)
    
    # Calculate sums
    sum_x = np.sum(x_values)
    sum_y = np.sum(y_values)
    sum_xy = np.sum(x_values * y_values)
    sum_x_square = np.sum(x_values ** 2)
    sum_y_square = np.sum(y_values ** 2)

    # Pearson correlation formula
    numerator = (n * sum_xy) - (sum_x * sum_y)
    denominator = np.sqrt((n * sum_x_square - sum_x**2) * (n * sum_y_square - sum_y**2))

    if denominator == 0:
        return 0  
    
    return numerator / denominator 

def kesimpulan_dan_validasi(rxy, rtabel):
    # Membuat kesimpulan berdasarkan rtabel
    kesimpulan = 'valid' if rxy > rtabel else 'tidak valid'
    
    # Menentukan tingkat validasi berdasarkan nilai rxy
    if abs(rxy) >= 0.8:
        validasi = 'Sangat Kuat'
    elif abs(rxy) >= 0.6:
        validasi = 'Kuat'
    elif abs(rxy) >= 0.4:
        validasi = 'Sedang'
    elif abs(rxy) >= 0.2:
        validasi = 'Lemah'
    else:
        validasi = 'Sangat Lemah'
    
    return kesimpulan, validasi



hasil = []


# Menghitung Pearson correlation, kesimpulan, dan validasi untuk setiap kolom
for col in ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']:
    rxy = pearson_correlation_fixed(df, col, 'yi')
    kesimpulan, validasi = kesimpulan_dan_validasi(rxy, rtabel)
    
    hasil.append({
            # 'Pertanyaan': col,
            'rxy': rxy,
            'rtabel': rtabel,
            'Kesimpulan': kesimpulan,
            'Koefisien': validasi
        })

# Mengonversi list hasil ke dalam DataFrame
df_hasil = pd.DataFrame(hasil).T



df_hasil

,0,1,2,3,4,5,6,7,8,9
rxy,0.672768,0.651505,0.617826,0.614816,0.614458,0.727059,0.697113,0.65192,0.635586,0.617147
rtabel,0.514,0.514,0.514,0.514,0.514,0.514,0.514,0.514,0.514,0.514
Kesimpulan,valid,valid,valid,valid,valid,valid,valid,valid,valid,valid
Koefisien,Kuat,Kuat,Kuat,Kuat,Kuat,Kuat,Kuat,Kuat,Kuat,Kuat


In [19]:
import pandas as pd
import numpy as np


df = pd.read_excel('./dataset/kuesioner.xlsx')

# Function untuk menghitung Cronbach Alpha 
def cronbach_alpha_from_df(df):
    # Menghilangkan kolom 'DM' yang berisi identifier, kita hanya ambil kolom pertanyaan
    data = df.drop(columns=['DM']).values

    # Jumlah item (kolom) dalam data (P1 sampai P10)
    k = data.shape[1]
    print(f"Jumlah item (k): {k}")
    
    # Step 1: Menghitung varians untuk setiap item (kolom)
    variances = []
    for i in range(k):
        item_variance = np.var(data[:, i], ddof=1)
        variances.append(item_variance)
        print(f"Varians item P{i+1} (s^2y): {item_variance}")
        
    
    
    # Step 2: Menghitung varians total dari skor keseluruhan
    total_item_variance = sum(variances)
    print(f"Total keseluruhan varians item: {total_item_variance}")
    total_scores = np.sum(data, axis=1)
    total_variance = np.var(total_scores, ddof=1)
    
    print(f"Varians total (s^2x): {total_variance}")
    
    # Step 3: Menghitung Cronbach's Alpha
    alpha = (k / (k - 1)) * (1 - sum(variances) / total_variance)
    
    print(f"Cronbach's Alpha: {alpha}")
    return alpha

# Menghitung Cronbach's Alpha dari DataFrame
cronbach_alpha_from_df(df)




Jumlah item (k): 10
Varians item P1 (s^2y): 0.45714285714285713
Varians item P2 (s^2y): 0.49523809523809514
Varians item P3 (s^2y): 1.0666666666666667
Varians item P4 (s^2y): 0.780952380952381
Varians item P5 (s^2y): 0.5523809523809524
Varians item P6 (s^2y): 0.5999999999999999
Varians item P7 (s^2y): 0.6380952380952382
Varians item P8 (s^2y): 0.6380952380952383
Varians item P9 (s^2y): 0.6666666666666666
Varians item P10 (s^2y): 0.8380952380952381
Total keseluruhan varians item: 6.733333333333333
Varians total (s^2x): 27.83809523809524
Cronbach's Alpha: 0.8423613486904625


0.8423613486904625